# Deep Q-Learning 

For this assignment we will implement the Deep Q-Learning algorithm with Experience Replay as described in breakthrough paper __"Playing Atari with Deep Reinforcement Learning"__. We will train an agent to play the famous game of __Breakout__.

In [1]:
import sys
import gym
import torch
import pylab
import random
import numpy as np
from collections import deque
from datetime import datetime
from copy import deepcopy
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from utils import *
from agent import *
from model import *
from config import *
%matplotlib inline
%load_ext autoreload
%autoreload 2

## Understanding the environment

In the following cell, we initialise our game of __Breakout__ and you can see how the environment looks like. For further documentation of the of the environment refer to https://gym.openai.com/envs. 

In [2]:
env = gym.make('BreakoutDeterministic-v4')
env.render()

True

In [3]:
number_lives = find_max_lifes(env)
state_size = env.observation_space.shape
action_size = 3
rewards, episodes = [], []

## Creating a DQN Agent

Here we create a DQN Agent. This agent is defined in the __agent.py__. The corresponding neural network is defined in the __model.py__. 

__Evaluation Reward__ : The average reward received in the past 100 episodes/games.

__Frame__ : Number of frames processed in total.

__Memory Size__ : The current size of the replay memory.

In [4]:
agent = Agent(action_size)
evaluation_reward = deque(maxlen=evaluation_reward_length)
frame = 0
memory_size = 0

### Main Training Loop

In [5]:
for e in range(EPISODES):
    done = False
    score = 0

    history = np.zeros([5, 84, 84], dtype=np.uint8)
    step = 0
    d = False
    state = env.reset()
    life = number_lives

    get_init_state(history, state)

    while not done:
        step += 1
        frame += 1
        if render_breakout:
            env.render()

        # Select and perform an action
        action = agent.get_action(np.float32(history[:4, :, :]) / 255.)

        
        next_state, reward, done, info = env.step(action + 1)

        frame_next_state = get_frame(next_state)
        history[4, :, :] = frame_next_state
        terminal_state = check_live(life, info['ale.lives'])

        life = info['ale.lives']
        r = np.clip(reward, -1, 1)

        # Store the transition in memory 
        agent.memory.push(deepcopy(frame_next_state), action, r, terminal_state)
        # Start training after random sample generation
        if(frame >= train_frame):
            agent.train_policy_net(frame)
            # Update the target network
            if(frame % Update_target_network_frequency)== 0:
                agent.update_target_net()
        score += reward
        history[:4, :, :] = history[1:, :, :]

        if frame % 50000 == 0:
            print('now time : ', datetime.now())
            rewards.append(np.mean(evaluation_reward))
            episodes.append(e)
            pylab.plot(episodes, rewards, 'b')
            pylab.savefig("./save_graph/breakout_dqn.png")

        if done:
            evaluation_reward.append(score)
            # every episode, plot the play time
            print("episode:", e, "  score:", score, "  memory length:",
                  len(agent.memory), "  epsilon:", agent.epsilon, "   steps:", step,
                  "    evaluation reward:", np.mean(evaluation_reward))

            # if the mean of scores of last 10 episode is bigger than 400
            # stop training
            if np.mean(evaluation_reward) > 10:
                torch.save(agent.model, "./save_model/breakout_dqn")
                sys.exit()

episode:0  score:0.0  memory length:122  epsilon:1.0   steps:122    evaluation reward:0.0
episode:1  score:1.0  memory length:311  epsilon:1.0   steps:189    evaluation reward:0.5
episode:2  score:9.0  memory length:691  epsilon:1.0   steps:380    evaluation reward:3.3333333333333335
episode:3  score:1.0  memory length:878  epsilon:1.0   steps:187    evaluation reward:2.75
episode:4  score:5.0  memory length:1211  epsilon:0.9997901200000046   steps:333    evaluation reward:3.2
episode:5  score:2.0  memory length:1410  epsilon:0.9995931100000088   steps:199    evaluation reward:3.0
episode:6  score:2.0  memory length:1624  epsilon:0.9993812500000134   steps:214    evaluation reward:2.857142857142857
episode:7  score:0.0  memory length:1758  epsilon:0.9992485900000163   steps:134    evaluation reward:2.5
episode:8  score:2.0  memory length:1971  epsilon:0.9990377200000209   steps:213    evaluation reward:2.4444444444444446
episode:9  score:1.0  memory length:2162  epsilon:0.9988486300000

RuntimeError: Expected 4-dimensional input for 4-dimensional weight 32 4 8, but got 3-dimensional input of size [4, 84, 84] instead